# COLX 535 Lab Assignment 2: Working with CFG Parsers

## Assignment Objectives

In this assignment you will 
- Use the Stanford CoreNLP parser to parse new text into constituency trees
- Create a parsing gold standard and use it to evaluate parsers
- Build a context-free grammar from existing parses (optional assignment)

### How does this relate to class?

In class, we're discussing CFGs (Context-free grammars).  In this lab, you will be evaluating the quality of the Stanford Parser, and then using it to create and modify your own CFG (and feature grammar), to establish a treebank of your own.

## Getting Started

This assignment requires that you have set up the Stanford parser. First, make sure you have the more recent version of [Java](https://www.java.com/en/download/), then get the [Stanford CoreNLP](https://stanfordnlp.github.io/CoreNLP/) package. Make sure that you get the 4.3.2 version (or at least >=4.2.0).  (It's probably easiest to download from [HuggingFace]( https://huggingface.co/stanfordnlp/CoreNLP/tree/main))

To load Stanford CoreNLP in Python, change the `coreNLP_dir` variable in the code below to where you unzipped Stanford coreNLP. You can follow this [tutorial](https://bbengfort.github.io/snippets/2018/06/22/corenlp-nltk-parses.html). Once the coreNLP server is running, you will be able to access it through NLTK.

There are 3 steps:

* Install Java (the JRE is fine for this assignment, but you may want to consider the JDK if you want to do Java development)
* Set an environment variable called JAVA_HOME to the location of your java.exe file (you might need to restart your machine after setting this variable)
* Download the coreNLP, unzip it, and point to it in the code below.

It may take a few seconds or up to a minute to start the server.

In [1]:
from nltk.parse.corenlp import CoreNLPServer
import os
import time

coreNLP_dir = os.path.join('models', 'stanford-corenlp-4.5.8')

# # Create the server
# server = CoreNLPServer(
#    os.path.join(coreNLP_dir, "stanford-corenlp-4.5.8.jar"),
#    os.path.join(coreNLP_dir, "stanford-corenlp-4.5.8-models.jar")    
# )

# # Start the server in the background
# server.start()

In [2]:
coreNLP_dir

'models\\stanford-corenlp-4.5.8'

run following in terminal:

```ba
java -mx4g -cp 'models\\stanford-corenlp-4.5.8/*' edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000
```

Try to parse a sentence to make sure that everything works.

In [3]:
from nltk.parse.corenlp import CoreNLPParser

parser = CoreNLPParser()
parse = next(parser.raw_parse("I put the book in the box on the table."))
print(parse)

(ROOT
  (S
    (NP (PRP I))
    (VP
      (VBD put)
      (NP (DT the) (NN book))
      (PP (IN in) (NP (DT the) (NN box)))
      (PP (IN on) (NP (DT the) (NN table))))
    (. .)))


In [4]:
list(parser.raw_parse('I put the book in the box on the table.'))[0].pretty_print()

                         ROOT                              
                          |                                 
                          S                                
  ________________________|______________________________   
 |                        VP                             | 
 |    ____________________|________________              |  
 |   |       |            PP               PP            | 
 |   |       |         ___|____         ___|___          |  
 NP  |       NP       |        NP      |       NP        | 
 |   |    ___|___     |    ____|___    |    ___|____     |  
PRP VBD  DT      NN   IN  DT       NN  IN  DT       NN   . 
 |   |   |       |    |   |        |   |   |        |    |  
 I  put the     book  in the      box  on the     table  . 



In [5]:
parse = next(parser.raw_parse("They gave me yellow and blue pants"))
print(parse)

(ROOT
  (S
    (NP (PRP They))
    (VP
      (VBD gave)
      (NP (PRP me))
      (NP (ADJP (JJ yellow) (CC and) (JJ blue)) (NNS pants)))))


In [6]:
list(next(parser.raw_parse("They gave me yellow and blue pants")))[0].pretty_print()

      S                                 
  ____|_________                         
 |              VP                      
 |     _________|__________              
 |    |    |               NP           
 |    |    |           ____|_________    
 NP   |    NP        ADJP            |  
 |    |    |     _____|________      |   
PRP  VBD  PRP   JJ    CC       JJ   NNS 
 |    |    |    |     |        |     |   
They gave  me yellow and      blue pants



Run the code below if you want to shut down the coreNLP server after you've finished with it. It's a good idea to shut down the parser after finishing work with it because it may remain running in the background and you may not be able to start another parser instance without restarting your computer or manually killing the parser process. 

If you forget to stop the server, next time when you try to launch it, you'll get an error. In this case, you may first need to kill the old server manually. To do this, you can run `ps -ax | grep stanford` on the commandline (at least on OSX and Linux) which should give you the process ID of the server, e.g. 11111. You can then use `kill -9 11111` to kill the parser, after which you should be able to start the server again.  On Windows, you should be able to use the Task Manager to kill the process.

In [7]:
try:
    server.stop()
except:
    pass

Other things you'll need:

In [8]:
import nltk
from nltk.tree import Tree
from nltk.grammar import CFG, Nonterminal, Production, FeatureGrammar
from nltk.parse import CoreNLPParser
from nltk.corpus import brown

nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\ArnoZ\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

## Tidy Submission

rubric={mechanics:1}

To get the marks for tidy submission:

- Submit the assignment by filling in this jupyter notebook with your answers embedded
- Be sure to follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions)

### Exercise 1: Test the parser

#### 1.1
rubric={accuracy:2}

Get the Stanford parser working, then parse the first 20 sentences of the NLTK `movie_reviews` corpus, and report the average depth (height) of the parse trees. If you find the parser is failing to parse something, you can skip over it.

You should retain the tokenization in the `movie_reviews` corpus. You can use `parser.parse()` to parse the tokenized input sentences. Note that `parser.parse()` returns an iterator over possible parses. There may be several if the sentence is ambiguous. You can compute statistics on the first sentence which is returned by `parser.parse()` (ie, don't calculate an average height over all parses - we're just interested in the height of the first parse, for simplicity.)

In [9]:
from nltk.corpus import movie_reviews

# your code here

movie_reviews_sentences = list(movie_reviews.sents())[:20]
heights = []

for i, sentence in enumerate(movie_reviews_sentences):
    
    try:
        parse_tree = next(parser.raw_parse(" ".join(sentence)))
        h = parse_tree.height()
        print(f"Height {i + 1}: {h}")
        heights.append(h)
    except Exception as e:
        print(f"Failed to parse: {' '.join(sentence)}\nError: {e}")

average_height = sum(heights) / len(heights)
print(f"\nAverage height: {average_height}")

Height 1: 10
Height 2: 7
Height 3: 12
Height 4: 7
Height 5: 8
Height 6: 4
Height 7: 4
Height 8: 12
Height 9: 20
Height 10: 12
Height 11: 8
Height 12: 9
Height 13: 17
Height 14: 12
Height 15: 15
Height 16: 15
Height 17: 10
Height 18: 5
Height 19: 24
Height 20: 23

Average height: 11.7


### Exercise 2: Build a gold standard

One typical way to build treebanks is, rather than having humans build a tree from scratch, instead use an automatic parser to give an initial parse, and then have humans do a second pass to fix any errors. That's what you're going to do in this exercise. 

We will use the following three sentences from the NLTK `movie_reviews` corpus to build a mini-treebank:

In [10]:
corpus = [
    ['oh', ',', 'and', 'by', 'the', 'way', ',', 'this', 'is', 
     'not', 'a', 'horror', 'or', 'teen', 'slasher', 'flick', '.'],
    
    ['little', 'do', 'they', 'know', 'the', 'power', 'within', '.'],
    
    ['so', ',', 'if', 'robots', 'and', 'body', 'parts', 'really', 
     'turn', 'you', 'on', ',', 'here', "'", 's', 'your', 'movie', '.']
]

In each of the three sections below you will see a parse tree produced by CoreNLPParser. Each of the trees contains at least one parse error. You'll see a brief informal description of the error and it is your task to fix the tree.  

Create an NLTK Tree corresponding to the correct parse, which should be appended to the `gold_standard_parses` list below. You can do this manually by printing the tree, creating a triple-quoted string, modifying it, and converting it back into a `Tree` using the function `Tree.fromstring` following the example below. If you are unsure exactly how to correct something, read through the lecture slides. Many common parse errors are explained there.

In [11]:
# Example:

# The second phrase should be a VP, not an NP
err_tree_str = '''(S
(NP (NNS Dogs)) 
(NP (VBN bark))
(. .))
'''

corr_tree_str = '''(S
(NP (NNS Dogs)) 
(VP (VBN bark))
(. .))
'''

corr_tree = Tree.fromstring(corr_tree_str)
corr_tree.pretty_print()

      S      
  ____|____   
 NP   VP   | 
 |    |    |  
NNS  VBN   . 
 |    |    |  
Dogs bark  . 



In [12]:
# Store your corrected nltk.Tree objects in this list
gold_standard_parses = []

#### Sentence 1
rubric={accuracy:2}

In [13]:
# The word "flick" should be modified by the entire noun phrase 
# "a horror or teen slasher" instead of just "teen slasher". A noun 
# phrase which modifies a noun is labeled as NML.

err_tree_str = '''(ROOT
  (S
    (INTJ (UH oh))
    (, ,)
    (CC and)
    (PP (IN by) (NP (DT the) (NN way)))
    (, ,)
    (NP (DT this))
    (VP
      (VBZ is)
      (RB not)
      (NP
        (NP (DT a) (NN horror))
        (CC or)
        (NP (NML (NN teen) (NN slasher)) (NN flick))))
    (. .)))'''

# your code here

corr_tree_str = '''(ROOT
  (S
    (INTJ (UH oh))
    (, ,)
    (CC and)
    (PP (IN by) (NP (DT the) (NN way)))
    (, ,)
    (NP (DT this))
    (VP
      (VBZ is)
      (RB not)
      (NP
        (NML
          (NP (DT a) (NN horror))
          (CC or)
          (NP (NN teen) (NN slasher)))
        (NN flick)))
    (. .)))'''

corr_tree = Tree.fromstring(corr_tree_str)
gold_standard_parses.append(corr_tree)
corr_tree.pretty_print()

                                      ROOT                                                  
                                       |                                                     
                                       S                                                    
  _____________________________________|__________________________________________________   
 |    |   |       |           |   |                      VP                               | 
 |    |   |       |           |   |     _________________|_________                       |  
 |    |   |       |           |   |    |    |                      NP                     | 
 |    |   |       |           |   |    |    |                   ___|_________________     |  
 |    |   |       PP          |   |    |    |                 NML                    |    | 
 |    |   |    ___|___        |   |    |    |        __________|________             |    |  
INTJ  |   |   |       NP      |   NP   |    |       NP         | 

#### Sentence 2
rubric={accuracy:2}

In [14]:
# The PP "within" should attach to the NP "the power", not to the VP "know the power". 

err_tree_str = '''(ROOT
  (S
    (NP (RB little))
    (VP
      (VBP do)
      (SBAR
        (S
          (NP (PRP they))
          (VP (VBP know) (NP (DT the) (NN power)) (PP (IN within))))))
    (. .)))'''

# your code here

corr_tree_str = '''(ROOT
  (S
    (NP (RB little))
    (VP
      (VBP do)
      (SBAR
        (S
          (NP (PRP they))
          (VP
            (VBP know)
            (NP
              (NP (DT the) (NN power))
              (PP (IN within)))))))
    (. .)))'''

corr_tree = Tree.fromstring(corr_tree_str)
gold_standard_parses.append(corr_tree)
corr_tree.pretty_print()

                ROOT                          
                 |                             
                 S                            
   ______________|__________________________   
  |              VP                         | 
  |      ________|____                      |  
  |     |            SBAR                   | 
  |     |             |                     |  
  |     |             S                     | 
  |     |    _________|____                 |  
  |     |   |              VP               | 
  |     |   |     _________|____            |  
  |     |   |    |              NP          | 
  |     |   |    |          ____|_____      |  
  NP    |   NP   |         NP         PP    | 
  |     |   |    |     ____|____      |     |  
  RB   VBP PRP  VBP   DT        NN    IN    . 
  |     |   |    |    |         |     |     |  
little  do they know the      power within  . 



#### Sentence 3
rubric={accuracy:2}

In [15]:
# There are several errors."body" and "parts" should form an NP. 
# Moreover, "here's your movie" should form a clause and "s" is in 
# fact a verb.  

err_tree_str = '''(ROOT
  (SINV
    (ADVP (RB so))
    (, ,)
    (PP
      (IN if)
      (NP (NML (NNS robots) (CC and) (NN body)) (NNS parts)))
    (ADVP (RB really))
    (VP
      (VBP turn)
      (NP (NP (PRP you)) (PP (IN on) (, ,) (NP (RB here))) ('' '))
      (S (NP (POS s))))
    (NP (PRP$ your) (NN movie))
    (. .)))'''

# your code here

corr_tree_str = '''(ROOT
  (SINV
    (ADVP (RB so))
    (, ,)
    (PP
      (IN if)
      (NP (NML (NNS robots) (CC and) (NN body) (NNS parts))))
    (ADVP (RB really))
    (VP
      (VBP turn)
      (NP
        (NP (PRP you))
        (PP
          (IN on)
          (, ,)
          (S
            (NP (EX here))
            (VP (VBZ 's)
                (NP (PRP$ your) (NN movie))))))
      )
    (. .)))'''

corr_tree = Tree.fromstring(corr_tree_str)
gold_standard_parses.append(corr_tree)
corr_tree.pretty_print()

                                          ROOT                                            
                                           |                                               
                                          SINV                                            
  _________________________________________|____________________________________________   
 |    |        |                     |          VP                                      | 
 |    |        |                     |      ____|___                                    |  
 |    |        |                     |     |        NP                                  | 
 |    |        |                     |     |     ___|_______                            |  
 |    |        |                     |     |    |           PP                          | 
 |    |        |                     |     |    |    _______|________                   |  
 |    |        PP                    |     |    |   |   |            S               

### Exercise 3: Evaluating parsers

Now that we have a gold standard, we can use it to evaluate parser output. 

#### 3.1
rubric={accuracy:3,quality:1}

Start by writing a function, get_constituents, which takes a parse tree and returns a set of tuples, where each tuple is (*label*, *start*, *end*) where *start* and *end* correspond to the indices of a corresponding constituent (phrase) in the sentence and *label* is the label of that constituent. 

Do **not** include simple POS constituents `(POS word)` like `(VBD ate)`. We want to evaluate the parser only on actual phrases.

**HINT:** You may want to use recursion to solve this assignment.

In [16]:
def get_constituents(tree: Tree, start_index: int=0) -> set:
    constituents = set()

    # your code here
    
    def traverse_with_root(node: Tree, start_index: int):
        # Add the current node to constituents 
        # if it has children or it is the final single-branch node
        if len(node) > 1 or (len(node) == 1 and isinstance(node[0], Tree)):
            end_index = start_index + len(node.leaves())
            constituents.add((node.label(), start_index, end_index))

        # Recursively traverse children
        current_index = start_index
        for child in node:
            if isinstance(child, Tree):
                traverse_with_root(child, current_index) # recursion
                current_index += len(child.leaves())
            else:
                current_index += 1

    traverse_with_root(tree, start_index)
    return constituents

In [17]:
tree = Tree.fromstring('''(S (NP (DT the) (DT mouse)) (VP (VBD ate) (NP (NP (DT the) (DT mouse)) (POS 's) (NN cheese))) )''')
assert get_constituents(tree) == {("S",0,7), ("NP",0,2), ("VP",2,7),("NP",3,7),("NP",3,5)}
tree = Tree.fromstring('''(S (NP (DET the) (NP (NN cat) (CC and) (NN dog))) (VP (VBD fought)))''')
assert get_constituents(tree) == {("S",0,5), ("NP",0,4), ("NP",1,4),("VP",4,5)}
print("Success!")

Success!


#### 3.2
rubric={accuracy:2,efficiency:1}

Write a function `parse_f1` which uses get_constituents to implement the constituent F-score measure discussed in the lecture and reading. It should be given two lists, a lists of proposed parses and a corresponding list of gold standard parses, and return an F-score reflecting how close the proposed parses match. For full points, you should keep a running count of the relevant numbers over the entire set, and not average f-score across the individual sentences. 

**Hint:** to get the efficiency point, you should take advantage of Python's fast set operations

In [18]:
def parse_f1(proposed_parses: list, gold_parses: list) -> float:
    f1score = 0
    
    # your code here

    # Initialize counts for precision and recall
    true_positives = 0
    total_proposed = 0
    total_gold = 0

    # Loop through the parses
    for proposed_tree, gold_tree in zip(proposed_parses, gold_parses):
        proposed_constituents = get_constituents(proposed_tree)
        gold_constituents = get_constituents(gold_tree)

        # Count overlaps (true positives)
        true_positives += len(proposed_constituents & gold_constituents)
        # Count total proposed and gold constituents
        total_proposed += len(proposed_constituents)
        total_gold += len(gold_constituents)

    # Calculate precision, recall, and F1 score
    precision = true_positives / total_proposed if total_proposed > 0 else 0
    recall = true_positives / total_gold if total_gold > 0 else 0
    f1score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1score

In [19]:
gold_parses = [Tree.fromstring('''(S (NP (DT the) (DT mouse)) (VP (VBD ate) (NP (NP (DT the) (DT mouse)) (POS 's) (NN cheese))) )'''), Tree.fromstring('''(S (NP (NNS mice)) (VP (VBD love) (NP (NNS ducks))))''')]
proposed_parses = [Tree.fromstring('''(S (NP (DT the) (DT mouse)) (VP (VBD ate) (NP (NP (DT the) (DT mouse)) (POS 's) (NN cheese))) )'''), Tree.fromstring('''(S (NP (NNS mice) (NN love)) (VP (VBZ ducks)))''')]
assert 0.71 > parse_f1(proposed_parses, gold_parses) > 0.7
print("Success!")

Success!


#### 3.3

rubric={accuracy:1}

Parse your three example sentences from assignment 2 using CoreNLPParser (you can find the sentences in the list `corpus`), extract the constituents and evaluate the result against `gold_standard_parses`. 

**Hint:** Your F1-score should be > 0.6 (the actual score may depend a bit on your version of the CoreNLP parser).

In [20]:
# your code here

# Parse sentences
proposed_parses = [next(parser.raw_parse(" ".join(sentence))) for sentence in corpus]

# Calculate F1 score
f1 = parse_f1(proposed_parses, gold_standard_parses)
f1

0.684931506849315

#### 3.4

rubric={reasoning:1}

Given the way we build our gold standard, do you think this is a valid indication of the quality of parsers? Why or why not? What about if we tested the parser on the Penn Treebank corpus instead?

**Answer:**

I don't think it is valid due to these problems. Firstly, there are only three sentences, this insufficient in diversity of syntactic structures may cause subjectivity and bias. And the robustness and generalization ability of the evaluation are limited. Secondly, the F1 score only focuses on the complete match of components, instead of partial matches or semantic understanding. This may affect the actual performance of the parser. In contrast, using a standard corpus like Penn Treebank, is more reasonable for evaluation.

### Exercise 4: Generate a grammar

#### 4.1
rubric= {accuracy:1}

Parse trees implicitly contain the production rules for a CFG defined by the productions which are present in the parse tree. You can access these productions using the member function `nltk.Tree.productions()`. 

Produce a grammar corresponding to the CFG productions in your three sentences from exercise 2, and print it out. One "trick" to sort string in the manner you want is to create a sorting function that biases the sort by placing extra information (that will be sorted correctly) at the start of each string.  So you should write a sort function that does this, and pass it to the `sorted` function under the `key` parameter (similar to how we did lambda sorting in 521).

In [21]:
rules = set()

# your code here

# Sort the rules according to type: First ROOT rules, followed
# by S rules, other non-terminals and finally terminal rules.
# This is not a mandatory part of the assignment.

for tree in gold_standard_parses:
    rules.update(tree.productions())

# Custom sort function for rules
def sort_key(rule: nltk.grammar.Production) -> str:
    lhs = str(rule.lhs())
    if lhs == "ROOT":
        return "0" + lhs
    elif lhs == "S":
        return "1" + lhs
    elif lhs.isupper():
        return "2" + lhs
    else:
        return "3" + lhs

# Sort rules and print
sorted_rules = sorted(rules, key=sort_key)
for rule in sorted_rules:
    print(rule)

ROOT -> S
ROOT -> SINV
S -> NP VP .
S -> NP VP
S -> INTJ , CC PP , NP VP .
ADVP -> RB
CC -> 'or'
CC -> 'and'
DT -> 'this'
DT -> 'a'
DT -> 'the'
EX -> 'here'
IN -> 'within'
IN -> 'if'
IN -> 'by'
IN -> 'on'
INTJ -> UH
NML -> NP CC NP
NML -> NNS CC NN NNS
NN -> 'flick'
NN -> 'body'
NN -> 'movie'
NN -> 'teen'
NN -> 'slasher'
NN -> 'power'
NN -> 'way'
NN -> 'horror'
NNS -> 'parts'
NNS -> 'robots'
NP -> PRP
NP -> DT
NP -> NP PP
NP -> EX
NP -> NN NN
NP -> DT NN
NP -> NML
NP -> RB
NP -> PRP$ NN
NP -> NML NN
PP -> IN , S
PP -> IN NP
PP -> IN
PRP -> 'they'
PRP -> 'you'
PRP$ -> 'your'
RB -> 'really'
RB -> 'little'
RB -> 'not'
RB -> 'so'
SBAR -> S
SINV -> ADVP , PP ADVP VP .
UH -> 'oh'
VBP -> 'do'
VBP -> 'turn'
VBP -> 'know'
VBZ -> "'s"
VBZ -> 'is'
VP -> VBZ RB NP
VP -> VBP NP
VP -> VBP SBAR
VP -> VBZ NP
, -> ','
. -> '.'


#### 4.2
rubric= {accuracy:1}

Show the rules in 4.1 are indeed sufficient to parse the sentences in the list `corpus`. Using an NLTK EarleyChartParser parser for this. Print out the number of parses for each sentence. Don't print out the parses themselves, as there might be a lot of them and you could crash your notebook (this depends a bit on how you fixed the parse trees in exercise 2). You should also set the `trace` keyword argument of the parser to 0 for the same reason. 

If you have individual sentences which are taking longer than 2 minutes to parse, you can skip over them.

In [22]:
from nltk import EarleyChartParser
from nltk.grammar import CFG

# your code here

# Add terminal rules for all unique words in the corpus
vocabulary = set()

for sentence in corpus:
    vocabulary.update(word.lower() for word in sentence)

for word in vocabulary:
    rules.add(Production(Nonterminal(word), [word]))

# Create a CFG grammar from the rules
grammar = CFG(Nonterminal('ROOT'), list(rules))

# Initialize an EarleyChartParser with the grammar
earley_parser = EarleyChartParser(grammar, trace=0)

# Parse each sentence in the corpus
for i, sentence in enumerate(corpus):
    sentence = " ".join(sentence)
    parses = list(earley_parser.parse(sentence.split()))
    print(f"Sentence {i + 1}: {len(parses)} parses")

Sentence 1: 1 parses
Sentence 2: 2 parses
Sentence 3: 0 parses


#### 4.3  Optional
rubric= {accuracy:2}

Convert your CFG grammar into a feature grammar and implement noun-verb agreement (you should use the feature values `3SG` and `NON3SG`). Make sure that all S, NP and VP rules use agreement features. 

Give an example sentence, which displays noun-verb agreement. Show that your feature grammar can parse this sentence. Then create a version of the same sentence without proper agreement, and show that the number of parses for this setence is lower (possibly zero). 

Your grammar shouldn't contain any rules where the LHS contains special characters like `.` or `$`. Otherwise `FeatureGrammar.fromstring` might give an error. This means that you might need to rename some of your non-terminals. 

In [ ]:
# your code here
